In [1]:
import sys
sys.path.insert(0,'..')
import sib
import pandas as pd
import numpy as np

## Order of parameters

### Contacts:

in order: (i, j, t, lambda)

### Observations:
(i,s,t) which is in order:

- node
- state
- time

In [2]:
#name_folder_data = "ProximityModel/"
name_folder_data = "dummy/"
#root_folder_data = "../sibilla/notebooks/data/"
root_folder_data = "./data/"
folder_data = root_folder_data + name_folder_data

In [3]:
file_contacts_name = folder_data +  "transmissions.csv"
contacts_df = pd.read_csv(file_contacts_name, dtype={
                                         "i":np.int,
                                        "j":np.int,
                                        "lamb":np.float,
                                        "t":np.int
                                                })

In [4]:
contacts_df

,i,j,lambda,t
0,2,3,0.02,0
1,2,3,0.02,0
2,1,2,0.02,0
3,2,1,0.02,1
4,1,2,0.02,1
5,0,1,0.02,3


In [5]:
# set the correct order!
contacts_df = contacts_df[["i", "j","t", "lambda"]] 

#removing symmetric contacts
d = {'i':'j','j':'i'}
contacts_df = pd.concat([contacts_df, contacts_df.rename(columns=d)]).drop_duplicates()
contacts_df = contacts_df.sort_values("t")
contacts_df = contacts_df[contacts_df["i"] < contacts_df["j"]]

# time must start from zero
assert(contacts_df["t"].iloc[0] == 0)

contacts = list(contacts_df.to_records(index=False))

tot_contacts = len(contacts);
print(f"number of unique contacts: {tot_contacts}")

number of unique contacts: 4


In [6]:
file_obs_name = folder_data +  "observations.csv"
obs_df = pd.read_csv(file_obs_name,
                    dtype={"s":np.int,
                           "i":np.int,
                           "t_test":np.int})
obs = obs_df[["i","s","t_test"]]
observations = list(obs.to_records(index=False))
tot_obs = len(observations)
print(f"number of observartions {tot_obs}")

number of observartions 2


In [7]:
import math
mu=0.5;
params = sib.Params(prob_r=sib.PriorDiscrete([math.exp(-mu*x) for x in range(50)]), pseed=0.01)
params

Params(prob_i=Uniform(1),prob_r=PriorDiscrete(1,0.606531,0.367879,0.22313,0.135335,0.082085,0.0497871,0.0301974,0.0183156,0.011109,0.00673795,0.00408677,0.00247875,0.00150344,0.000911882,0.000553084,0.000335463,0.000203468,0.00012341,7.48518e-05,4.53999e-05,2.75364e-05,1.67017e-05,1.01301e-05,6.14421e-06,3.72665e-06,2.26033e-06,1.37096e-06,8.31529e-07,5.04348e-07,3.05902e-07,1.85539e-07,1.12535e-07,6.8256e-08,4.13994e-08,2.511e-08,1.523e-08,9.23745e-09,5.6028e-09,3.39827e-09,2.06115e-09,1.25015e-09,7.58256e-10,4.59906e-10,2.78947e-10,1.6919e-10,1.02619e-10,6.22414e-11,3.77513e-11,2.28973e-11),pseed=0.01,psus=0.5)

In [8]:
f = sib.FactorGraph(contacts=contacts, observations=observations, params=params)

In [9]:
def callback(t,err,f):
    print(t,err,end="\r")

In [10]:
sib.iterate(f, maxit=100,tol=1e-5,callback=callback)

In [11]:
sib.marginal(f.nodes[0])

[(0.8087238295242778, 0.1912761704757222, 0.0),
 (0.49954871535690654, 0.5004512846430935, 0.0),
 (0.0, 1.0, 0.0)]

In [12]:
contacts

[(2, 3, 0, 0.02), (1, 2, 0, 0.02), (1, 2, 1, 0.02), (0, 1, 3, 0.02)]

In [19]:
f.append_contact(3,2,12,0.2)

ValueError: time of contacts should be ordered

In [20]:
sib.iterate(f)

0 0.0
